In [ ]:
# ensure appropriate modules/widgets are activated
import io
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# check version of ipywidgets sinmply for reference to pypi.org (package index) if any errors encountered  
# working correctly as of 26-04-2025
import ipywidgets
print(ipywidgets.__version__)

In [ ]:
# ***Stage 1: Select Data Import Method***
user_choice = widgets.RadioButtons(
    options=['Import a file directly', 'Copy and paste data'],
    description='Select method:',
    disabled=False
)

# ***Stage 2: File Import Section***
file_upload = widgets.FileUpload(
    accept='.csv, .xlsx',
    multiple=False,
    description='Select File'
)
file_output = widgets.Output()

def handle_file_upload(change):
    with file_output:
        file_output.clear_output()
        print("File upload triggered!")
        if file_upload.value:
            try:
                # Check if the value is a dictionary or tuple and extract the first uploaded file
                if isinstance(file_upload.value, dict):
                    uploaded_file = next(iter(file_upload.value.values()))
                elif isinstance(file_upload.value, tuple):
                    uploaded_file = file_upload.value[0]
                else:
                    print("Unexpected file_upload.value type:", type(file_upload.value))
                    return

                # Now handle the uploaded_file. In many cases it will be a dict 
                # with keys 'name' and 'content'
                if isinstance(uploaded_file, dict):
                    fname = uploaded_file.get('name', 'Unknown file')
                    file_content = uploaded_file.get('content', b'')
                else:
                    # Fallback: if it is an object with attributes
                    fname = uploaded_file.name
                    file_content = uploaded_file.content

                # Wrap the content in a BytesIO stream for reading
                file_stream = io.BytesIO(file_content)
                try:
                    df = pd.read_csv(file_stream)
                except Exception as e_csv:
                    file_stream.seek(0)
                    try:
                        df = pd.read_excel(file_stream)
                    except Exception as e_excel:
                        print("Error reading file as CSV or Excel.")
                        print("CSV Error:", e_csv)
                        print("Excel Error:", e_excel)
                        return

                print(f"Loaded file: {fname}, size: {len(file_content)} bytes")
                display(df.head())
            except Exception as e:
                print("Error processing file upload:", e)

file_upload.observe(handle_file_upload, names='value')

upload_box = widgets.VBox([
    widgets.Label("Upload your CSV or Excel file:"),
    file_upload,
    file_output
])

# ***Stage 3: Copy-Paste Section***
paste_text = widgets.Textarea(
    placeholder='Paste your spreadsheet data here (for example, from Excel)',
    description='Data:',
    layout=widgets.Layout(width='80%', height='150px')
)
paste_button = widgets.Button(
    description='Create DataFrame',
    button_style='success'
)
paste_output = widgets.Output()

def create_df_from_paste(b):
    with paste_output:
        paste_output.clear_output()
        raw_text = paste_text.value.strip()
        if not raw_text:
            print("No data pasted!")
            return
        try:
            if ',' in raw_text:
                delimiter = ','
            elif '\t' in raw_text:
                delimiter = '\t'
            else:
                delimiter = ','
            df = pd.read_csv(io.StringIO(raw_text), sep=delimiter)
            print("DataFrame loaded from pasted data:")
            display(df.head())
        except Exception as e:
            print("Error creating DataFrame:", e)

paste_button.on_click(create_df_from_paste)

paste_box = widgets.VBox([
    widgets.Label("Paste your spreadsheet data:"),
    paste_text,
    paste_button,
    paste_output
])

# ***Stage 4: Display the Selected Method***
method_container = widgets.VBox()

def update_method_container(change):
    method_container.children = []
    if change['new'] == 'Import a file directly':
        method_container.children = [upload_box]
    elif change['new'] == 'Copy and paste data':
        method_container.children = [paste_box]

user_choice.observe(update_method_container, names='value')
user_choice.value = 'Import a file directly'
update_method_container({'new': user_choice.value})

# ***Stage 5: Final Interface Display***
display(widgets.VBox([user_choice, method_container]))

In [ ]:
# Optional debugging step if errors encountered: print the current file upload value (run both lines below).
#print("File Upload Value:")
#print(file_upload.value)

In [ ]:
def transform_to_dataframe():
    """
    Checks the selected data input method (file upload or copy-paste),
    transforms the provided data into a Pandas DataFrame assuming the first row
    contains headers, and then displays a confirmation output that includes:
      - the file name or a label (for pasted data)
      - the header (list of column names)
      - the first five rows as an exemplar

    Returns the DataFrame if successful, or None otherwise.
    """
    # File import path
    if user_choice.value == 'Import a file directly':
        if file_upload.value:
            try:
                # Extract the uploaded file in a robust way
                if isinstance(file_upload.value, dict):
                    uploaded_file = next(iter(file_upload.value.values()))
                elif isinstance(file_upload.value, tuple):
                    uploaded_file = file_upload.value[0]
                else:
                    print("Unexpected file upload value type:", type(file_upload.value))
                    return None

                # Get the file content and file name (if available)
                if isinstance(uploaded_file, dict):
                    fname = uploaded_file.get('name', 'Unknown file')
                    file_content = uploaded_file.get('content', b'')
                else:
                    fname = getattr(uploaded_file, 'name', 'Unknown file')
                    file_content = getattr(uploaded_file, 'content', b'')

                # Wrap content in a BytesIO stream (simulate a file object)
                file_stream = io.BytesIO(file_content)
                try:
                    df = pd.read_csv(file_stream, header=0)  # Assume first row is header
                except Exception as e_csv:
                    # If CSV reading fails, try Excel instead
                    file_stream.seek(0)
                    try:
                        df = pd.read_excel(file_stream, header=0)
                    except Exception as e_excel:
                        print("Error reading file as CSV or Excel.")
                        print("CSV Error:", e_csv)
                        print("Excel Error:", e_excel)
                        return None

                # Confirmation message along with header and first five rows of the DataFrame
                print(f"Headed DataFrame loaded from file '{fname}':")
                print("DataFrame columns (header):", list(df.columns))
                print("First five rows:")
                display(df.head())
                return df

            except Exception as e:
                print("Error processing file upload:", e)
                return None
        else:
            print("No file has been uploaded.")
            return None

    # Pasted data path
    elif user_choice.value == 'Copy and paste data':
        raw_text = paste_text.value.strip()
        if not raw_text:
            print("No data pasted!")
            return None
        try:
            # Determine the delimiter by checking text content
            if ',' in raw_text:
                delimiter = ','
            elif '\t' in raw_text:
                delimiter = '\t'
            else:
                delimiter = ','  # Default to comma
            df = pd.read_csv(io.StringIO(raw_text), sep=delimiter, header=0)
            # Use a label to indicate the origin of this DataFrame
            print("Headed DataFrame loaded from pasted data:")
            print("DataFrame columns (header):", list(df.columns))
            print("First five rows:")
            display(df.head())
            return df
        except Exception as e:
            print("Error creating DataFrame:", e)
            return None

# Run the transformation function to display the exemplar.
transform_to_dataframe()

In [ ]:
'''
You should now have a dataframe created from either an import (large data) or copy and paste (small data) in a few seconds!
'''
print(transform_to_dataframe)